In [1]:
import pandas as pd
import networkx as nx
import json

file_path = "/Users/Mah/Desktop/Ass3/data_scopus.csv"  
df = pd.read_csv(file_path)

df_cleaned = df.dropna(subset=['Year', 'Authors', 'Authors with affiliations'])

df_cleaned['Publisher'] = df_cleaned['Publisher'].fillna('')
df_cleaned['Year'] = df_cleaned['Year'].fillna('')
df_cleaned = df_cleaned[
    ~df_cleaned['Year'].isin(['Unknown', '']) &
    ~df_cleaned['Authors'].str.contains('Unknown', na=False) &
    ~df_cleaned['Authors with affiliations'].str.contains('Unknown', na=False)
]

G = nx.Graph()

for _, row in df_cleaned.iterrows():
    authors = row['Authors'].split(", ")
    affiliations = row['Authors with affiliations'].split("; ") if pd.notna(row['Authors with affiliations']) else []
    
    if pd.isna(row['Year']) or row['Year'] in ['', 'Unknown']:
        continue
    if pd.isna(row['Publisher']) or row['Publisher'] in ['', 'Unknown']:
        continue
    if not any(affiliations):  
        continue
    
    for author in authors:
        G.add_node(author, type='author', year=row['Year'], publisher=row['Publisher'], affiliation="")
    
    for affiliation in affiliations:
        if affiliation not in G:
            G.add_node(affiliation, type='affiliation')
        for author in authors:
            G.add_edge(author, affiliation)
            if G.nodes[author]['affiliation'] == "":
                G.nodes[author]['affiliation'] = affiliation

nodes_to_remove = [
    node for node, attrs in G.nodes(data=True)
    if any(attrs.get(attr) in ['', 'Unknown'] for attr in ['year', 'publisher', 'affiliation'])
]
G.remove_nodes_from(nodes_to_remove)

for _, row in df_cleaned.iterrows():
    authors = row['Authors'].split(", ")
    for i, author1 in enumerate(authors):
        for author2 in authors[i + 1:]:
            if G.has_node(author1) and G.has_node(author2):
                G.add_edge(author1, author2)

data = nx.node_link_data(G)

for node in data['nodes']:
    for field in ['publisher', 'year', 'affiliation']:
        if node.get(field) in ['', 'Unknown']:
            del node[field]
    
    node['label'] = node['id']
    
output_file = "cleaned_graph_data.json"
with open(output_file, 'w') as f:
    json.dump(data, f, indent=4)

print(f"Graph data saved to {output_file}")

Graph data saved to cleaned_graph_data.json


/var/folders/b1/9kxdfvz90r1dt1tzhtkyv20h0000gp/T/ipykernel_17604/145397356.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Publisher'] = df_cleaned['Publisher'].fillna('')
/var/folders/b1/9kxdfvz90r1dt1tzhtkyv20h0000gp/T/ipykernel_17604/145397356.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Year'] = df_cleaned['Year'].fillna('')
